[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório.
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/ml_intro/class_02/Laborat%C3%B3rios/lab_utils_ml_intro_week_2.zip
!unzip -n -q lab_utils_ml_intro_week_2.zip

# Engenharia de Recursos e Regressão Polinomial

![](./images/C1_W2_Lab07_FeatureEngLecture.PNG)


## Objetivos
Neste laboratório, você irá:
- Explorar a engenharia de recursos e a regressão polinomial, que permite usar o mecanismo de regressão linear para ajustar funções muito complicadas e até mesmo extremamente não lineares.

## Ferramentas
Você utilizará a função desenvolvida nos laboratórios anteriores, bem como o Matplotlib e o NumPy. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lab_utils_multi import zscore_normalize_features, run_gradient_descent_feng
np.set_printoptions(precision=2)  # precisão de exibição reduzida em matrizes numéricas

<a name='FeatureEng'></a>
# Visão geral da Engenharia de Recursos e da Regressão Polinomial

Resumidamente, a regressão linear fornece um meio de criar modelos do tipo:
$$f_{\mathbf{w},b} = w_0x_0 + w_1x_1+ ... + w_{n-1}x_{n-1} + b \tag{1}$$ 
E se seus recursos/dados não forem lineares ou forem combinações de recursos? Por exemplo, os preços das moradias não tendem a ser lineares com a área habitável, mas penalizam casas muito pequenas ou muito grandes, resultando nas curvas mostradas no gráfico acima. Como podemos usar o mecanismo de regressão linear para ajustar essa curva? Lembre-se de que o "mecanismo" que temos é a capacidade de modificar os parâmetros $\mathbf{w}$, $\mathbf{b}$ em (1) para "ajustar" a equação aos dados de treinamento. No entanto, nenhum ajuste dos parâmetros $\mathbf{w}$,$\mathbf{b}$ em (1) conseguirá um ajuste a uma curva não linear.


<a name='PolynomialFeatures'></a>
## Recursos Polinomiais

Acima, estávamos considerando um cenário em que os dados eram não lineares. Vamos tentar usar o que sabemos até agora para ajustar uma curva não linear. Começaremos com uma quadrática simples: $y = 1+x^2$

Usaremos [`np.c_[...]`](https://numpy.org/doc/stable/reference/generated/numpy.c_.html), que é uma rotina do NumPy para concatenar ao longo do limite da coluna.

In [ ]:
# criar dados
x = np.arange(0, 20, 1)
y = 1 + x**2
X = x.reshape(-1, 1)

model_w,model_b = run_gradient_descent_feng(X,y,iterations=1000, alpha = 1e-2)

plt.scatter(x, y, marker='x', c='r', label="Valor Real"); plt.title("sem engenharia de recursos")
plt.plot(x,X@model_w + model_b, label="Valor Predito");  plt.xlabel("X"); plt.ylabel("y"); plt.legend(); plt.show()

Bem, como esperado, não é um bom ajuste. O que é necessário é algo como $y= w_0x_0^2 + b$, ou um **recurso polinomial**.
Para conseguir isso, você pode modificar os *dados de entrada* para *projetar* os recursos necessários. Se você trocar os dados originais por uma versão que eleve o valor de $x$ ao quadrado, poderá obter $y= w_0x_0^2 + b$. Vamos tentar. Troque `X` por `X**2` abaixo:

In [ ]:
# criar dados
x = np.arange(0, 20, 1)
y = 1 + x**2

# Engenharia de recursos
X = x**2      #<-- recurso criado com engenharia de recursos

In [ ]:
X = X.reshape(-1, 1)  #X deve ser um matriz 2-D
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha = 1e-5)

plt.scatter(x, y, marker='x', c='r', label="Valor Real"); plt.title("Recurso x**2 adicionado")
plt.plot(x, np.dot(X,model_w) + model_b, label="Valor Predito"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

Ótimo! Ajuste quase perfeito. Observe os valores de $\mathbf{w}$ e b impressos logo acima do gráfico: `w,b encontrados por descida de gradiente: w: [1.], b: 0.0490`. A descida do gradiente modificou nossos valores iniciais de $\mathbf{w},b $ para que fossem (1.0,0.049) ou um modelo de $y=1*x_0^2+0,049$, muito próximo de nossa meta de $y=1*x_0^2+1$. Se você o executasse por mais tempo, a correspondência poderia ser melhor. 

### Selecionando Recursos
<a name='GDF'></a>
Acima, sabíamos que era necessário um termo de $x^2$. Talvez nem sempre seja óbvio quais recursos são necessários. É possível adicionar uma variedade de recursos potenciais para tentar encontrar o mais útil. Por exemplo, e se, em vez disso, tivéssemos tentado: $y=w_0x_0 + w_1x_1^2 + w_2x_2^3+b$ ? 

Execute as próximas células. 

In [ ]:
# criar dados
x = np.arange(0, 20, 1)
y = x**2

# Engenharia de recursos
X = np.c_[x, x**2, x**3]      #<-- recurso criado com engenharia de recursos

In [ ]:
model_w,model_b = run_gradient_descent_feng(X, y, iterations=10000, alpha=1e-7)

plt.scatter(x, y, marker='x', c='r', label="Valor Real Value"); plt.title("recursos x, x**2, x**3")
plt.plot(x, X@model_w + model_b, label="Valor Predito"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

Observe que o valor de $\mathbf{w}$, `[0.08 0.54 0.03]` e b é `0.0106`. Isso implica que o modelo após o ajuste/treinamento é:
$$ 0,08x + 0,54x^2 + 0,03x^3 + 0,0106 $$
O gradiente descendente enfatizou os dados que melhor se ajustam aos dados de $x^2$ aumentando o termo $w_1$ em relação aos outros.  Se você executasse por um período muito longo, ele continuaria a reduzir o impacto dos outros termos. 
>O gradiente descendente está escolhendo os recursos "corretos" para nós, enfatizando o parâmetro associado

Vamos revisar essa ideia:
- Um valor de peso menor implica em um recurso menos importante/correto e, no extremo, quando o peso se torna zero ou muito próximo de zero, o recurso associado não é útil para ajustar o modelo aos dados.
- Acima, após o ajuste, o peso associado ao recurso $x^2$ é muito maior do que os pesos para $x$ ou $x^3$, pois é o mais útil para ajustar os dados. 

### Uma visão alternativa
Acima, os recursos polinomiais foram escolhidos com base em sua correspondência com os dados de destino. Outra maneira de pensar sobre isso é observar que ainda estamos usando a regressão linear depois de criarmos novos recursos. Sendo assim, os melhores recursos serão lineares em relação ao alvo. Isso é mais bem compreendido com um exemplo. 

In [ ]:
# criar dados
x = np.arange(0, 20, 1)
y = x**2

# Engenharia de recursos
X = np.c_[x, x**2, x**3]      #<-- recurso criado com engenharia de recursos
X_features = ['x','x^2','x^3']

In [ ]:
fig,ax=plt.subplots(1, 3, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X[:,i],y)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("y")
plt.show()

Acima, fica claro que o recurso $x^2$ mapeado em relação ao valor-alvo $y$ é linear. A regressão linear pode, então, gerar facilmente um modelo usando esse recurso.

### _Feature Scaling_
Conforme descrito no último laboratório, se o conjunto de dados tiver recursos com escalas significativamente diferentes, deve-se aplicar o _feature scaling_ para acelerar a descida do gradiente. No exemplo acima, há $x$, $x^2$ e $x^3$ que, naturalmente, terão escalas muito diferentes. Vamos aplicar a normalização com _z-score_ ao nosso exemplo.

In [ ]:
# criar dados
x = np.arange(0,20,1)
X = np.c_[x, x**2, x**3]
print(f"Faixa de pico a pico por coluna nos dados originais        X:{np.ptp(X,axis=0)}")

# adicionar normalização
X = zscore_normalize_features(X)     
print(f"Faixa de pico a pico por coluna em X normalizado:{np.ptp(X,axis=0)}")

Agora podemos tentar novamente com um valor mais agressivo de alfa:

In [ ]:
x = np.arange(0,20,1)
y = x**2

X = np.c_[x, x**2, x**3]
X = zscore_normalize_features(X) 

model_w, model_b = run_gradient_descent_feng(X, y, iterations=100000, alpha=1e-1)

plt.scatter(x, y, marker='x', c='r', label="Valor Real"); plt.title("Recursos x x**2, x**3 normalizados")
plt.plot(x,X@model_w + model_b, label="Valor Predito"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()

O _feature scaling_ permite que a convergência seja muito mais rápida.   
Observe novamente os valores de $\mathbf{w}$. O termo $w_1$, que é o termo $x^2$, é o mais enfatizado. A descida de gradiente praticamente eliminou o termo $x^3$.

### Funções Complexas
Com a engenharia de recursos, até mesmo funções bastante complexas podem ser modeladas:

In [ ]:
x = np.arange(0,20,1)
y = np.cos(x/2)

X = np.c_[x, x**2, x**3,x**4, x**5, x**6, x**7, x**8, x**9, x**10, x**11, x**12, x**13]
X = zscore_normalize_features(X) 

model_w,model_b = run_gradient_descent_feng(X, y, iterations=1000000, alpha = 1e-1)

plt.scatter(x, y, marker='x', c='r', label="Valor Real"); plt.title("Recursos x x**2, x**3 normalizados")
plt.plot(x,X@model_w + model_b, label="Valor Predito"); plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()



## Parabéns!
Neste laboratório, você:
- aprendeu como a regressão linear pode modelar funções complexas e até mesmo altamente não lineares usando a engenharia de recursos
- reconheceu que é importante aplicar o dimensionamento de recursos ao fazer a engenharia de recursos